In Lasagna AI, an **agent** is a unit of AI-powered reasoning that performs _specific_ work. Agents are the building blocks of your AI system — you compose simple agents together to create powerful multi-agent workflows.

In [1]:
# This page will use the following imports:

from lasagna import Model, EventCallback, AgentRun
from lasagna import (
    recursive_extract_messages,
    extract_last_message,
    override_system_prompt,
    flat_messages,
    parallel_runs,
    chained_runs,
    extraction,
)
from lasagna import known_models
from lasagna.tui import tui_input_loop

import os
import re
from enum import Enum
from pydantic import BaseModel

from dotenv import load_dotenv

## What is an _Agent_?

A piece of software is an "agent" if it displays some sort of "agency." Circular, yes, so let's keep going...

### What is _Agency_?

_Agency_ is the ability to act on one's own behalf. That is, software has _agency_ if it is allowed to _decide_ and _act_ on its own.

- Software that computes π? Not much agency. It does, and always will do, one thing.
- Software that organizes your calendar without your input? Lots of agency!

### What is an _AI Agent_?

The phrase "AI Agent" has risen in popularity since ~2024. Typically, when people use this phrase, they mean a piece of software that uses a Large Language Model (LLM\*) and has some _tool calling_ capability so that it can _affect_ the world (e.g. send emails, query for today's news, organize your calendar, etc.)

This is consistent with the idea of _agency_ from above. An LLM, on its own, has no _agency_ (it just spits tokens at you). If you connect that _same_ LLM to software functions (via _tool calling_), then suddenly the LLM gains the ability to _act_, and people start calling it "agentic".

### How does _Lasagna AI_ define an Agent?

Everything above ☝️ is too theoretical. When the rubber meets the road, what actually _is_ an **agent** inside _Lasagna AI_?

In _Lasagna_, an **agent** is a unit of work that leverages a `Model`.

Think of an agent as a specialized _worker_ that:

1. **Analyzes** the current situation.
2. **Decides** what needs to be done.
3. **Acts** using AI models and tools.
4. **Records** its output.

In Lasagna, agents are _composable_. You begin by developing individual agents, each with a narrow focus, then you _compose_ them together into a complex multi-agent system. Like humans, it's helpful to decompose and delegate tasks amongst the group.

In the next section, we'll see how to write **agents** using Lasagna AI's interfaces.

## The Agent Interface

Every Lasagna agent follows the same pattern — it's a _callable_ (either a function or a callable-object) that takes exactly **three parameters**:

In [2]:
async def my_agent(
    model: Model,                   # ← the AI model available to this agent
    event_callback: EventCallback,  # ← used for streaming and event handling
    prev_runs: list[AgentRun],      # ← previous work, context, or conversation history
) -> AgentRun:
    # Agent logic goes here...
    raise RuntimeError('not yet implemented')

Let's understand what each parameter represents:

- **`model`**: The AI model (like GPT-4o, Claude, etc.) that your agent can use for reasoning, text generation, or decision-making.
- **`event_callback`**: A function for handling streaming events and progress updates. This enables real-time feedback as your agent works.
- **`prev_runs`**: The history of previous work. In a conversation, this contains past messages. In a workflow, this contains results from earlier steps.

The agent returns an `AgentRun` — a structured representation of what the agent generated.

## How do I write an agent?

When you sit down to write an agent, here is what you must consider:

### 1. Analyze the Current Situation

Your agent must _examine_ `prev_runs` to understand what has happened so far. It may find:

- previous messages in a conversation,
- results from earlier agents in a workflow, and/or
- intermediate outputs from a multi-step process.

### 2. Make Behavioral Decisions

Your agent must _decide_ what to do next. It might:

- generate a response using the AI `model`, or
- use the AI `model` to extract data, or
- pass tools to the AI `model`, or
- split its task into multiple subtasks and delegate those to downstream agents, or
- do _many_ of the things above as many times as it chooses!

### 3. Take Action

Your agent must _execute_ its decision:

- **Model interaction**: Invoke the AI `model` to generate text, reason about problems, or extract structured outputs.
- **Tool usage**: Send emails, query a database, etc.
- **Agent delegation**: Invoke downstream agents to handle subtasks.

### 4. Record its Output

Your agent must _construct_ and _return_ an `AgentRun` that contains:

- any new information that it generated, and/or
- results from sub-agents it coordinated.

## Real Agent Examples

Let's look at some **real** examples to see agents in action.

### Setup

Before we write and run some agents, we need to set up our "binder" (see the [quickstart guide](../quickstart.ipynb) for what this is).

In [3]:
load_dotenv()

if os.environ.get('OPENAI_API_KEY'):
    print('Using OpenAI')
    binder = known_models.openai_gpt_4o_binder

elif os.environ.get('ANTHROPIC_API_KEY'):
    print('Using Anthropic')
    binder = known_models.anthropic_claude_sonnet_4_binder

else:
    assert False, "Neither OPENAI_API_KEY nor ANTHROPIC_API_KEY is set! We need at least one to do this demo."

Using OpenAI


### The Conversational Agent

This is the simplest type of agent — it uses the message history to generate a new text response:

In [4]:
async def chat_agent(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    # Extract all previous messages from the conversation:
    messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)

    # Use the model to generate a _new_ response:
    new_messages = await model.run(event_callback, messages, tools=[])

    # Wrap the new messages into an `AgentRun` result:
    return flat_messages('chat_agent', new_messages)

In [5]:
await tui_input_loop(binder(chat_agent))   # type: ignore[top-level-await]

>  Hi, my name is Ryan.


Hi Ryan! Nice to meet you. How can I help you today?


>  What is my name?


Your name is Ryan! 😊


>  exit


### The Specialist Agent

Agents can be specialized for particular tasks. Here's an agent that focuses on providing helpful coding advice:

In [6]:
CODING_SYSTEM_PROMPT = """
You are a helpful coding assistant.
Provide clear, practical advice with code examples when appropriate.
Focus on best practices and explain your reasoning.
""".strip()

In [7]:
async def coding_advisor(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    # Extract all previous messages from the conversation:
    messages = recursive_extract_messages(prev_runs, from_tools=False, from_extraction=False)

    # Generate a response with an OVERRIDDEN system prompt!
    modified_messages = override_system_prompt(messages, CODING_SYSTEM_PROMPT)
    new_messages = await model.run(event_callback, modified_messages, tools=[])

    # Wrap the new messages into an `AgentRun` result:
    return flat_messages('coding_advisor', new_messages)

In [8]:
await tui_input_loop(binder(coding_advisor))   # type: ignore[top-level-await]

>  Who are you? (answer briefly)


I’m a coding assistant here to help you with programming questions, code problems, and learning best practices. How can I help?


>  How do I add numbers in Python? (answer in 1 sentence)


To add numbers in Python, use the `+` operator, like `result = num1 + num2`.


>  exit


### The Information Extractor

Let's make an agent that does **structured output** to _extract_ information from the user's message. In particular, we'll have this agent classify the user's message (i.e. it is "extracting" the classification, if you will).

In [9]:
INTENT_CLASSIFIER_SYSTEM_PROMPT = """
Your job is to classify the user's message into one of the following categories:
 - `small_talk`: Comments like "hi", "how are you?", etc.
 - `programming`: Questions or comments about programming languages, libraries, etc.
 - `other`: Any message that is not small talk and not programming.
""".strip()

# In a production-grade system, you'd probably expand your system prompt to
# be more thorough; we're going for minimal here to keep this demo short.

class Category(Enum):
    small_talk = 'small_talk'
    programming = 'programming'
    other = 'other'

class CategoryOutput(BaseModel):
    thoughts: str
    category: Category

In [10]:
async def intent_classifier(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    # Get **ONLY** the last message from the conversation so far:
    #    (Just for demo-purposes, to show you can do whatever you want with
    #     `prev_runs` 😁. A production-grade intent classifier would consider
    #     more than just the last message.)
    last_message = extract_last_message(prev_runs, from_tools=False, from_extraction=False)

    # Generate a structured output response with an OVERRIDDEN system prompt!
    messages = override_system_prompt([last_message], INTENT_CLASSIFIER_SYSTEM_PROMPT)
    new_message, result = await model.extract(event_callback, messages, CategoryOutput)
    assert isinstance(result, CategoryOutput)

    # Wrap the new messages into an `AgentRun` result:
    return extraction('intent_classifier', [new_message], result)

In [11]:
await tui_input_loop(binder(intent_classifier))   # type: ignore[top-level-await]

>  Hi!


CategoryOutput({"thoughts":"The user is greeting with a simple 'Hi!', which is small talk.","category":"small_talk"})



>  Sup?


CategoryOutput({"thoughts":"The message is a casual greeting.","category":"small_talk"})



>  What is Python?


CategoryOutput({"thoughts":"The user is asking about Python, which is a programming language.","category":"programming"})



>  What is 2+2?


CategoryOutput({"thoughts":"The user is asking a basic math question.","category":"other"})



>  exit


### The 'Back on Track' Agent

This agent is pretty useless on its own, but you'll see soon why we're creating it. It just tells the user to get back on track!

In [12]:
BACK_ON_TRACK_SYSTEM_PROMPT = """
The user's message has been deemed to be off-topic.
Please politely tell them that their message is off-topic.
Do not respond to their question or their request. Just politely
tell them they are off-topic and need to return to the topic
at-hand.
""".strip()

In [13]:
async def back_on_track(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    last_message = extract_last_message(prev_runs, from_tools=False, from_extraction=False)
    messages = override_system_prompt(
        [last_message],
        BACK_ON_TRACK_SYSTEM_PROMPT,
    )
    return flat_messages(
        'back_on_track',
        await model.run(event_callback, messages, tools=[]),
    )

In [14]:
await tui_input_loop(binder(back_on_track))   # type: ignore[top-level-await]

>  Hi!


Hello! It seems your message is off-topic. Let's return to the topic at hand. How can I assist you with that?


>  exit


### The Routing Agent

Now, we'll put all the pieces together by making a **routing agent** that uses **all four** agents above!

In [15]:
async def router(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    # Decide which downstream agent to use, based on the user's intent:
    bound_intent_classifier = binder(intent_classifier)
    classification_run = await bound_intent_classifier(event_callback, prev_runs)
    assert classification_run['type'] == 'extraction'
    classification_result = classification_run['result']
    assert isinstance(classification_result, CategoryOutput)
    if classification_result.category == Category.small_talk:
        downstream_agent = binder(chat_agent)
    elif classification_result.category == Category.programming:
        downstream_agent = binder(coding_advisor)
    else:
        downstream_agent = binder(back_on_track)

    # Delegate to the downstream agent!
    downstream_run = await downstream_agent(event_callback, prev_runs)

    # Wrap *everything* that happened above into the return:
    return chained_runs('router', [classification_run, downstream_run])

In [16]:
await tui_input_loop(binder(router))   # type: ignore[top-level-await]

>  Hi!


CategoryOutput({"thoughts":"The user just greeted me, which is considered small talk.","category":"small_talk"})
Hello! 😊 How can I assist you today?


>  What is Python? (answer briefly)


CategoryOutput({"thoughts":"The question is directly about Python, a programming language, so it fits the programming category.","category":"programming"})
Python is a high-level, general-purpose programming language known for its simplicity, readability, and versatility. It supports multiple programming paradigms, including procedural, object-oriented, and functional programming. Python is widely used for web development, data science, artificial intelligence, automation, and much more, thanks to its extensive standard library and vibrant ecosystem of third-party packages.


>  What is 2+2?


CategoryOutput({"thoughts":"The message is a general, non-programming question related to arithmetic.","category":"other"})
I'm sorry, but your message is off-topic. Please return to the topic at hand.


>  How are you today? (also what is 2+2)


CategoryOutput({"thoughts":"The message contains a small talk element but also includes a simple math question which doesn't relate to programming.","category":"small_talk"})
I'm just a program, so I don’t have feelings, but thank you for asking! 😊 As for your question:  
**2 + 2 = 4**! 🎉 Let me know if you have more questions!


>  exit


### The Task Splitter

Another common task is for an agent to split work and delegate to multiple downstream agents. Let's do that next!

We'll use a silly example, for simplicity and brevity, where we'll split the user's message into individual sentences, then prompt an AI `model` one-at-a-time on each individual sentence. While this is a silly example, it shows how you can split up a problem for multiple downstream subagents.

In [17]:
async def splitter(
    model: Model,
    event_callback: EventCallback,
    prev_runs: list[AgentRun],
) -> AgentRun:
    # We'll only look at the most recent message:
    last_message = extract_last_message(prev_runs, from_tools=False, from_extraction=False)
    assert last_message['role'] == 'human'
    assert last_message['text']

    # We'll split the most recent message into sentences:
    #    (This is **not** a robust way to do it, but we're keeping the demo simple.)
    sentences = re.split(r'[\.\?\!] ', last_message['text'])
    sentences_as_agentruns = [
        flat_messages('splitter', [{
            'role': 'human',
            'text': sentence,
        }])
        for sentence in sentences
    ]

    # Have the `chat_agent` respond to each sentence:
    #    (Again, not particularly useful, but good for a brief demo.)
    bound_chat_agent = binder(chat_agent)
    downstream_runs: list[AgentRun] = []
    for task_input in sentences_as_agentruns:
        this_run = await bound_chat_agent(event_callback, [task_input])
        downstream_runs.append(this_run)

    # Wrap *everything* that happened above into the return:
    return parallel_runs('splitter', downstream_runs)

In [18]:
await tui_input_loop(binder(splitter))   # type: ignore[top-level-await]

>  Hi. What's up? How are you? What's 2+2?


Hello! How can I assist you today? 😊Not much, just here and ready to help! What's on your mind? 😊I'm just a virtual assistant, so I don't have feelings, but thanks for asking! How can I assist you today?2 + 2 = 4


>  Thanks! What did I just say?


You're welcome! Do you need help with anything? 😊I can't recall or access past interactions in this chat. Could you please repeat or clarify what you said?


>  exit


### Do anything!

It's up to you how to write your multi-agent AI system. You can mix-and-match ideas, include lots of behaviors in a _single_ agent, or split up tasks among _multiple_ agents. You can have "meta agents" that plan work for other agents, or "meta meta agents" that plan work for your "meta agents". As long as it is _safe_ and _works_, go for it!

## Why This Design?

Lasagna's agent design provides several key benefits:

### 🔌 **Pluggability** 

Every agent follows the same interface, so you can:

- swap one agent for another,
- combine agents from different sources, and
- test agents in isolation.

### 🥞 **Layering**

You can compose agents at any level:

- Use simple agents as building blocks.
- Combine them into more complex workflows.
- Build entire systems from agent compositions.

### 🔄 **Reusability**

Write an agent once, use it everywhere:

- as a standalone agent,
- as part of a larger workflow, or
- as a specialist in a multi-agent system.

## Next Steps

Now that you understand what agents are and how they work conceptually, you're ready to dive deeper into the technical details.

In the [next section](type_agentrun.ipynb), we'll explore the `AgentRun` data structure in detail — the standardized format that enables all this agent composition and layering.

You'll learn about:

- The four types of `AgentRun`.
- How to work with the recursive data structure.
- Helper functions for common patterns.
- Advanced features like cost tracking and serialization.

For more advanced agent patterns and real-world examples, check out:

- [Tool Use](../agent_features/tools.ipynb) — Agents that interact with external systems
- [Structured Output](../agent_features/structured_output.ipynb) — Agents that extract structured data
- [Layering](../agent_features/layering.ipynb) — Complex multi-agent compositions